In [1]:
import tensorflow as tf
import os

import glob

import nibabel
import numpy as np

/home/act65/anaconda3/envs/mri/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def read(fname):
    img = nibabel.load(fname)
    # can use variable size. but creates problems later...
    return np.array(img.get_fdata(), dtype=np.float32)[:, :, :100]  

basedir = '/home/act65/Downloads/ds108_R2.0.0/'
fnames = [str(fname) for fname in glob.iglob(basedir+'**/*T1w.nii.gz', recursive=True)]
def get_images():
    return (read(fname) for fname in fnames)

In [3]:
ds = tf.data.Dataset.from_generator(get_images, 
                                    output_shapes=[256, 256, 100],
                                    output_types=tf.float32)
ds = ds.map(lambda x: tf.expand_dims(tf.transpose(x, [2,0,1]), -1))
ds = ds.batch(10)
iterator = tf.data.Iterator.from_structure(ds.output_types,
                                           ds.output_shapes)
x = iterator.get_next()
init = iterator.make_initializer(ds)

In [4]:
def cnn(x, depth=3, width=8):
    with tf.variable_scope('cnn', reuse=tf.AUTO_REUSE):
        for i in range(depth):
            x = tf.layers.conv3d(
                x,
                width,
                6,
                strides=(2 if i in [0,1] else 1, 4, 4),
                padding='SAME',
                activation=tf.nn.selu,
                name=str(i))
    return x

def dcnn(x, depth=3, width=8):
    with tf.variable_scope('dcnn', reuse=tf.AUTO_REUSE):
        for i in range(depth):
            x = tf.layers.conv3d_transpose(
                x,
                width,
                6,
                strides=(2 if i in [0,1] else 1, 4, 4),
                padding='SAME',
                activation=tf.nn.selu,
                name=str(i))
        return tf.layers.conv3d_transpose(
                    x,
                    1,
                    1,
                    strides=(1, 1, 1),
                    padding='SAME',
                    name=str(i+1))

In [5]:
h = cnn(x)
y = dcnn(h)
print(h.shape, y.shape)

(?, 25, 4, 4, 8) (?, 100, 256, 256, 1)


In [6]:
beta = 1.0
loss = tf.losses.mean_squared_error(x,y)
loss += beta*tf.reduce_mean(tf.abs(h))  # sparse hidden space
with tf.variable_scope('opt', reuse=tf.AUTO_REUSE):
    train_step = tf.train.AdamOptimizer().minimize(loss)

summaries = tf.summary.merge([
    tf.summary.image('x', x),
    tf.summary.image('y', y),
    tf.summary.scalar('loss', loss),
]) 

In [7]:
!rm -rf /tmp/mri-cnn3d/0/

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(init)
    writer = tf.summary.FileWriter(graph=sess.graph, logdir='/tmp/mri-cnn3d/0')
    
    for i in range(10000):
        print(i)
        _, summ = sess.run([train_step, summaries])
        
        if i % 10 == 0:
            writer.add_summary(summ, i)
            

0


In [ ]:
# not happening on my computer...